In [ ]:
import csv
import json
import logging
import pandas as pd
import requests
import re
from github import Github
from github.GithubException import GithubException

RQ1 When do developer share the conversation links? Which part of PR timeline?
Note: In the paper, we reported our findings of recent events, in one of the 7 events: commented, PR created, review requested, ready for review, merged, closed, review request removed. In the following code, these 7 events are identified as "recent event group". Please refer to the "recent_event_group" column in the dfs.

In [ ]:
GITHUB_TOKEN = ""
g = Github(GITHUB_TOKEN)

In [ ]:
df = pd.read_excel("rq1_pr_english_only_no_404.xlsx")
len(df)

In [ ]:
df.head(10)

In [ ]:
df_comments = df[df["MentionedProperty"] == "comments.body"]
len(df_comments)

In [ ]:
df_reviews = df[df["MentionedProperty"] == "reviews.body"]
len(df_reviews)

In [ ]:
df_body = df[df["MentionedProperty"] == "body"]
len(df_body)

In [ ]:
def extract_comment_id(url):
    match = re.search(r'#issuecomment-(\d+)', url)
    return match.group(1) if match else None

In [ ]:
comment_ids = []
for _, row in df_comments.iterrows():
    comment_id = extract_comment_id(row["comment_url"])
    comment_ids.append(comment_id)

In [ ]:
df_comments["comment_ids"] = comment_ids

In [ ]:
pr_comments_creation_time = []
def get_pr_comment_creation(repo_name, pull_number, comment_id):
    try:
        repo = g.get_repo(f"{repo_name}")
        #print("here")
        pull_request = repo.get_pull(pull_number)
        #print("here")
        comment = pull_request.get_issue_comment(comment_id)
        #print("here")
        pr_comments_creation_time.append(comment.created_at)
            
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            pr_comments_creation_time.append("None")
            print("error")

In [ ]:
get_pr_comment_creation("rom-py/rompy", 40, 1742286490)
pr_comments_creation_time

In [ ]:
for _, row in df_comments.iterrows():
    get_pr_comment_creation(row["RepoName"], int(row["PullNumber"]), int(row["comment_ids"]))

In [ ]:
len(pr_comments_creation_time)

In [ ]:
df_comments["creation_time"] = pr_comments_creation_time

In [ ]:
def extract_review_id(url):
    match = re.search(r'#discussion_r(\d+)', url)
    return match.group(1) if match else None

In [ ]:
review_ids = []
for _, row in df_reviews.iterrows():
    review_id = extract_review_id(row["comment_url"])
    review_ids.append(review_id)

In [ ]:
df_reviews["comment_ids"] = review_ids

In [ ]:
review_comments_creation_time = []
def get_pr_comment_creation(repo_name, pull_number, comment_id):
    try:
        repo = g.get_repo(f"{repo_name}")
        #print("here")
        pull_request = repo.get_pull(pull_number)
        #print("here")
        comment = pull_request.get_review_comment(comment_id)
        #print("here")
        review_comments_creation_time.append(comment.created_at)
            
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            review_comments_creation_time.append("None")
            print("error")

In [ ]:
for _, row in df_reviews.iterrows():
    #print(row["RepoName"])
    #print(int(row["PullNumber"]))
    #print(int(row["comment_ids"]))
    get_pr_comment_creation(row["RepoName"], int(row["PullNumber"]), int(row["comment_ids"]))

In [ ]:
len(review_comments_creation_time)

In [ ]:
df_reviews["creation_time"] = review_comments_creation_time

In [ ]:
pr_body_creation_time = []
def get_pr_comment_creation(repo_name, pull_number):
    try:
        repo = g.get_repo(f"{repo_name}")
        #print("here")
        pull_request = repo.get_pull(pull_number)
        #print("here")
        pr_body_creation_time.append(pull_request.created_at)
            
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            pr_body_creation_time.append("None")
            print("error")

In [ ]:
for _, row in df_body.iterrows():
    #print(row["RepoName"])
    #print(int(row["PullNumber"]))
    get_pr_comment_creation(row["RepoName"], int(row["PullNumber"]))

In [ ]:
len(pr_body_creation_time)

In [ ]:
df_body["creation_time"] = pr_body_creation_time

In [ ]:
creation_times = []
creation_times = pr_comments_creation_time + review_comments_creation_time + pr_body_creation_time
len(creation_times)

In [ ]:
df_with_creation_time = pd.concat([df_comments, df_reviews, df_body], ignore_index=True)
len(df_with_creation_time)

In [ ]:
df_with_creation_time

In [ ]:
df_sorted_creation_times = df_with_creation_time.sort_values(by='chat_id')
df_sorted_creation_times

In [ ]:
pr_events = []
def get_pr_timeline(repo_name, pull_number):  
    try:
        repo = g.get_repo(repo_name)
        pr = repo.get_pull(pull_number)
        issue = pr.as_issue()
        events = issue.get_timeline() 
        pr_events.append(events)
            
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            pr_events.append("None")
            print("error")

In [ ]:
for _, row in df.iterrows():
    get_pr_timeline(row["RepoName"], int(row["PullNumber"]))

In [ ]:
len(pr_events)

In [ ]:
for event in pr_events[44]:
    print(event)

In [ ]:
pr_events[44]

In [ ]:
creation_times[0]

In [ ]:
creation_times_sorted = []
for _, row in df_sorted_creation_times.iterrows():
    creation_times_sorted.append(row["creation_time"])

len(creation_times_sorted)

In [ ]:
creation_times_sorted

In [ ]:
from datetime import datetime

def find_event(pr_events, creation_times):
    result = []

    for pr_idx, pr in enumerate(pr_events):
        comment_time = creation_times[pr_idx]
        events = pr
        most_recent_event = None
        #print(pr_idx)

        if events is not None:
            for event in events:
                if event != "None" and comment_time != "None":
                    event_time = event.created_at
                    if event_time is not None:
                        if event_time < comment_time:
                            if most_recent_event is None or event_time > most_recent_event.created_at:
                                most_recent_event = event
        if most_recent_event is not None:
            result.append(most_recent_event.event)
        else:
            result.append("None")

    return result

In [ ]:
test1 = [pr_events[0]]
test2 = [creation_times_sorted[0]]

In [ ]:
recent_event_comment = find_event(test1, test2)
recent_event_comment

In [ ]:
recent_events_comment = find_event(pr_events, creation_times_sorted)
len(recent_events_comment)

In [ ]:
df["recent_event"] = recent_events_comment
df

In [ ]:
df.to_csv("rq1_recent_event.csv")

In [ ]:
#only_reactions_body = [entry['reaction'] for entry in recent_events_comment]
from collections import Counter
recent_events_comment_counts = Counter(recent_events_comment)
print(recent_events_comment_counts)

In [ ]:
def find_event_group(pr_events, creation_times):
    result = []
    groups = ["ready_for_review", "review_requested", "commented", "merged", "closed", "review_request_removed"]

    for pr_idx, pr in enumerate(pr_events):
        comment_time = creation_times[pr_idx]
        events = pr
        most_recent_event = None
        #print(pr_idx)

        if events is not None:
            #print(event.event)
            if True: #any(event.event in groups for event.event in events):
                for event in events:
                    if event != "None" and comment_time != "None":
                        event_time = event.created_at
                        if event_time is not None:
                            if event_time < comment_time and event.event in groups:
                                if most_recent_event is None or event_time > most_recent_event.created_at:
                                    most_recent_event = event
                            
                result.append(most_recent_event.event if most_recent_event else "None")
            else:
                result.append("commented")
        else:
            result.append("None")

    return result

In [ ]:
recent_events_group_comment = find_event_group(pr_events, creation_times_sorted)
len(recent_events_group_comment)

In [ ]:
recent_events_group_comment

In [ ]:
df["recent_event_group"] = recent_events_group_comment
df

In [ ]:
df.to_excel("df_recent_event_group.xlsx", engine='openpyxl', index=False)

In [ ]:
from collections import Counter
recent_events_comment_counts_group = Counter(recent_events_group_comment)
print(recent_events_comment_counts_group)

In [ ]:
def assign_recent_event_group(row):
    if row['recent_event'] == "None": 
        if row['MentionedProperty'] in ['reviews.body', 'comments.body']:
            return 'commented'
        elif row['MentionedProperty'] == 'body':
            return 'pr_created'
    else:
        #if row['recent_event'] == "None": 
        return row['recent_event_group']

In [ ]:
df['recent_event_group'] = df.apply(assign_recent_event_group, axis=1)
df

In [ ]:
def assign_recent_event_group_2(row):
    if row['recent_event_group'] == "None": 
        return 'commented'
    else:
        return row['recent_event_group']

In [ ]:
df['recent_event_group'] = df.apply(assign_recent_event_group_2, axis=1)
df

In [ ]:
recent_events_comment_counts_group = Counter(df['recent_event_group'])
print(recent_events_comment_counts_group)

In [ ]:
df.to_csv("rq1_recent_event_group.csv")

In [ ]:
state_distribution = df.groupby(['recent_event_group', 'states']).size().unstack(fill_value=0)

In [ ]:
import matplotlib.pyplot as plt
state_distribution.plot(kind='bar', stacked=True, figsize=(10, 7))
plt.xlabel('Recent event group')
plt.ylabel('Count')
plt.title('State Distribution over Recent Event Group')
plt.legend(title='State')
plt.show()